# Importing Library

In [1]:
import pandas as pd                 #import pandas with the alias pd
import numpy as np                  #import numpy with the alias np
import matplotlib.pyplot as plt     #impot matplotlib.pyplot with the alias plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import svm, feature_selection, linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import r2_score

# Part 1

## Data Cleaning and Transformation

### Reading File

In [8]:
df_diabetic_csv = pd.read_csv('diabetic_data.csv')

In [13]:
print(df_diabetic_csv.columns)

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')


## Data Visualisation

## Model Building

# Part 2

## Improved Model